In [1]:
import requests
import json
import pandas as pd
import seaborn as sns
import time

In [2]:
%%capture
requests.post('https://maker.ifttt.com/trigger/heroku_app/with/key/{KEY}', params={'value1':'DJIA Rankings'})

In [3]:
def get_json(ticker):
    output = {}
    output['ticker'] = ticker

    quote = requests.get('https://finnhub.io/api/v1/quote?symbol='+ticker+'&token={TOKEN}')
    quote_json = json.loads(str(quote.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['quote'] = quote_json
        
    aggr = requests.get('https://finnhub.io/api/v1/scan/technical-indicator?symbol='+ticker+'&resolution=D&token={TOKEN}')
    aggr_json = json.loads(str(aggr.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['technicalAnalysis'] = aggr_json

    senti = requests.get('https://finnhub.io/api/v1/news-sentiment?symbol='+ticker+'&token={TOKEN}')
    senti_json = json.loads(str(senti.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    output['sentiment'] = senti_json
        
    fins = requests.get('https://finnhub.io/api/v1/stock/metric?symbol='+ticker+'&metric=all&token={TOKEN}')
    fins_json = json.loads(str(fins.json()).replace('\'','"').replace('None','"None"').replace('True','"True"').replace('False','"False"'))
    fins_json = fins_json["metric"]
    output['financials'] = fins_json

    return json.dumps(output, indent=2)

In [4]:
def get_quote(ticker):
    quote = get_json(ticker)
    quote_json = eval(quote)
    
    close = quote_json['quote']['c']
    prevClose = quote_json.get('quote').get('pc')
    change = round((close/prevClose-1)*100, 2)
    signal = quote_json.get('technicalAnalysis').get('technicalAnalysis').get('signal')
    articlesLastWk = quote_json.get('sentiment').get('buzz').get('articlesInLastWeek')
    weeklyAvg = int(quote_json.get('sentiment').get('buzz').get('weeklyAverage')+0.5)
    newsScore = quote_json.get('sentiment').get('companyNewsScore')
    sectorAvgNewsScore = quote_json.get('sentiment').get('sectorAverageNewsScore')
    buzz = quote_json.get('sentiment').get('buzz').get('buzz')
    bullishPercent = round(quote_json.get('sentiment').get('sentiment').get('bullishPercent')*100, 2)
    sectorAvgBullishPercent = round(quote_json.get('sentiment').get('sectorAverageBullishPercent')*100, 2)
    
    return {'ticker':ticker, 'change':change,'close':close,'prevClose':prevClose,'signal':signal,'articlesLastWk':articlesLastWk,'weeklyAvg':weeklyAvg,'newsScore':newsScore,'sectorAvgNewsScore':sectorAvgNewsScore,'buzz':buzz,'bullishPercent':bullishPercent,'sectorAvgBullishPercent':sectorAvgBullishPercent}

In [5]:
df = pd.DataFrame(columns=['ticker','change','close','prevClose','signal','articlesLastWk','weeklyAvg','newsScore','sectorAvgNewsScore','buzz','bullishPercent','sectorAvgBullishPercent'])

for ticker in ['JPM', 'IBM', 'UNH', 'PG', 'BA', 'VZ', 'GS', 'INTC', 'HD', 'MCD', 'CSCO', 'V', 'MSFT', 'KO', 'AXP', 'AMGN', 'MRK', 'CVX', 'MMM', 'TRV', 'JNJ', 'DIS', 'HON', 'CAT', 'NKE', 'CRM', 'WBA', 'AAPL', 'DOW', 'WMT']:
    df = df.append(get_quote(ticker), ignore_index = True)
    time.sleep(3)
    
df = df.sort_values(by=['bullishPercent','newsScore','articlesLastWk','change'], ascending=False)
df = df.reset_index(drop=True)

In [6]:
def apply_color(value):
    try:
        if value < 0:
            color = 'red'
        elif value > 0:
            color = 'green'
        else:
            color = 'black'
    except:
        if value=='sell' or value=='strong sell':
            color = 'red'
        elif value=='buy' or value=='strong buy':
            color = 'green'
        else:
            color = 'black'
    return 'color: %s' % color

blue_gradient = sns.diverging_palette(10, 240, s=100, as_cmap=True)
purple_gradient = sns.light_palette("purple", as_cmap=True)
orange_gradient = sns.light_palette("orange", as_cmap=True)
(df.style
    .format({'change': '{:.3}%','close': '${:.5}','prevClose': '${:.5}','newsScore': '{:.3}','sectorAvgNewsScore': '{:.3}','buzz': '{:.3}','bullishPercent': '{:.4}%','sectorAvgBullishPercent': '{:.4}%'})
    .applymap(apply_color, subset=['change','signal'])
    .background_gradient(cmap=blue_gradient, subset=['bullishPercent'])
    .background_gradient(cmap=purple_gradient, subset=['articlesLastWk'])
    .background_gradient(cmap=orange_gradient, subset=['newsScore']))

,ticker,change,close,prevClose,signal,articlesLastWk,weeklyAvg,newsScore,sectorAvgNewsScore,buzz,bullishPercent,sectorAvgBullishPercent
0,AMGN,0.06%,$237.22,$237.08,buy,44,21,1.0,0.525,2.1,100.0%,61.95%
1,PG,0.06%,$129.03,$128.95,neutral,20,39,1.0,0.544,0.513,100.0%,65.04%
2,HD,1.7%,$278.82,$274.17,buy,15,19,1.0,0.539,0.8,100.0%,65.04%
3,UNH,-2.54%,$329.32,$337.89,sell,25,29,0.917,0.524,0.862,100.0%,62.1%
4,KO,0.49%,$49.01,$48.77,neutral,16,19,0.917,0.544,0.853,100.0%,65.04%
5,NKE,1.44%,$140.63,$138.63,neutral,15,20,0.917,0.544,0.75,100.0%,65.04%
6,CSCO,3.23%,$47.25,$45.77,strong buy,25,30,0.861,0.538,0.833,100.0%,66.38%
7,DIS,2.15%,$180.23,$176.43,buy,50,82,0.8,0.533,0.613,100.0%,63.78%
8,VZ,0.64%,$55.14,$54.79,neutral,31,42,0.722,0.538,0.734,100.0%,66.38%
9,WBA,0.76%,$49.27,$48.9,neutral,26,27,0.655,0.544,0.963,100.0%,65.04%
